In [1]:
print ("ok 1234" )

ok 1234


In [3]:
pwd


'c:\\Users\\tanmoy\\Documents\\wind\\RAG'

In [6]:
import os 
os.chdir("c:\\Users\\tanmoy\\Documents\\wind\\RAG\\data")



In [3]:
pwd

'c:\\Users\\tanmoy\\Documents\\wind\\RAG'

In [11]:
from langchain.document_loaders import JSONLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
# Extract text from json files
def load_json_file():
    """Load json files from a specified directory."""
    loader = JSONLoader(
    file_path=r"c:\Users\tanmoy\Documents\wind\RAG\data\incidentListCleaned.json",
    jq_schema=".result.[],.countOfIncidentsByStatus.count[],.howToResolveBook.incidentResolutionByincidentDescription[],.sizeOfTotalIncident",
    text_content=False,
    json_lines=False,
)
    documents = loader.load()
    return documents

In [14]:
extracted_data = load_json_file()

In [ ]:
import pandas as pd

# Convert extracted_data (list of Documents or dicts) to DataFrame
df = pd.DataFrame([doc.page_content if hasattr(doc, "page_content") else doc for doc in extracted_data])
df

In [16]:
df = df.drop_duplicates()
len(df)

105

In [17]:
cleaned_data = df.to_dict(orient="records")

In [18]:
incidentSizeFromSource = cleaned_data[-1]
last_obj_val = list(incidentSizeFromSource.values())[0]
last_obj_val = int(last_obj_val)
last_obj_val



68

In [19]:
def dict_to_text_onIncident_details(d):
    return (
        f"the incident number {d.get('incidentNumber', '')}\n"
        f" has description :{d.get('incidentDescription', '').lower()}\n"
        f" with current state : {d.get('stateOfTicket', '').lower()}\n"
        f" and is currently assigned to : {d.get('incidentAssignedTo', 'no one ').lower()}\n"
        f" and has priority of : {d.get('severity_priority', '').lower()}\n"
        f" and work notes provided fir this incident number is : {d.get('workNotes', '').lower()}\n"
        f" resolution provided for this incident number : {d.get('incidentNumber', '')} was: {d.get('howItWasResolved', '').lower()}\n"
        f".This incident number : {d.get('incidentNumber', '')} has the tag {d.get('incidentTag', '').lower()}\n"
    )

In [20]:
# ...existing code...
def dict_to_text_onIncidentByStateCount(d):
    return (
        f"count of incident number with state : {d.get('incidentState', '')}\n"
        f"is : {d.get('incidentByStateCount', '')}\n"
    )



In [21]:
def dict_to_text_incident_description_resolution(d):
    return (
        f"incident with description : {d.get('incidentDescription', '')}\n"
        f" was closed and fixed with these steps provided as : {d.get('incidentResolution', '')}\n"
    )



In [22]:
from langchain.schema import Document
import json

fixed_cleaned_data_incident = [json.loads(list(item.values())[0]) for item in cleaned_data[:last_obj_val]]



In [24]:
from langchain.schema import Document
import json

fixed_cleaned_data_stateCount = [json.loads(list(item.values())[0]) for item in cleaned_data[last_obj_val:last_obj_val+4]]



In [25]:
from langchain.schema import Document
import json

fixed_cleaned_data_resolution = [json.loads(list(item.values())[0]) for item in cleaned_data[last_obj_val+4:len(cleaned_data)-1]]


In [ ]:
extracted_data_incident = [
    Document(
        page_content=dict_to_text_onIncident_details(item),
        metadata={"incidentNumber": item.get("incidentNumber", "")}
    )
    for item in fixed_cleaned_data_incident
]
extracted_data_incident


In [ ]:
extracted_data_count = [Document(page_content=dict_to_text_onIncidentByStateCount(item)) for item in fixed_cleaned_data_stateCount]
extracted_data_count


In [ ]:
extracted_data_resolution_book = [Document(page_content=dict_to_text_incident_description_resolution(item)) for item in fixed_cleaned_data_resolution]
extracted_data_resolution_book


In [ ]:
all_extracted_data = extracted_data_incident + extracted_data_count + extracted_data_resolution_book
all_extracted_data

In [39]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata= doc.metadata
            )
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(all_extracted_data)
minimal_docs

In [41]:
print(f"Number of minimal_docs: {len(minimal_docs)}")

Number of minimal_docs: 104


In [82]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    texts_chunk = minimal_docs
    return texts_chunk

In [86]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 104


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-mpnet-base-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [5]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")


In [7]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

In [8]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc1 = Pinecone(api_key=pinecone_api_key)

In [51]:
pc1

In [ ]:
from pinecone import ServerlessSpec 

index_name = "incident-chatbot"

if not pc1.has_index(index_name):
    pc1.create_index(
        name = index_name,
        dimension=768,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc1.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
retriever =docsearch.as_retriever(search_type="mmr", 
                                  search_kwargs={
                    'k': 100,  # Number of documents to return
                    'fetch_k': 50,  # Number of documents to fetch before applying MMR
                    'lambda_mult': 0.5  # Controls diversity: 0.5 is balanced
                })

In [ ]:


from langchain_mistralai import ChatMistralAI
chatModel = ChatMistralAI(
        model="mistral-tiny",
        temperature=0.1,
        max_tokens=1000,
        api_key=MISTRAL_API_KEY)

chatModel


ChatMistralAI(client=<httpx.Client object at 0x000001F694821E50>, async_client=<httpx.AsyncClient object at 0x000001F694822850>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='mistral-tiny', temperature=0.1, max_tokens=1000, model_kwargs={})

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:




system_prompt = (
    "You are a Service Now incident management assistant for question-answering tasks based on service now tickets. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

# Few-shot examples
few_shot_examples = [
    ("human", "what is tag for the incident number INC0010001?"),
    ("ai", "The tag for the incident number INC0000010 is: Java."),
    ("human", "what is tag for the incident number INC0000060?"),
    ("ai", "The tag for the incident number INC0000020 is: Sys Network."),
]

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt)] + few_shot_examples + [("human", "{input}")]
)

In [ ]:


question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "give me all the details of incident number INC0000039"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "what is incidentTag for the incident number INC0000038?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "numbers of incidents with different states"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "give me list of all the incident numbers in ascending order along with status and priority. do not show duplicate tickets"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "give me the total count of incident numbers and the breakup?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "group incidents by state"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "what is the details of incident number INC0009009?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "tell me about INC0007001 and how was it resolved?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "tell me about INC0009004 and how was it resolved?"})
print(response["answer"])